<a href="https://colab.research.google.com/github/syno3/Data-science-portfolio/blob/dev/smsSpamClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
### importing the required modules

import tensorflow as tf
import pandas as pd
from tensorflow import keras

import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import utils
import csv
import pandas as pd

In [84]:
# get data files
#loading the data

!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2021-07-11 07:27:10--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2021-07-11 07:27:10 (12.3 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2021-07-11 07:27:10--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[========

In [85]:
## converting to dataFrame and Preprocessing

train_df = pd.read_csv(train_file_path, sep='\t', names=['Classification', 'Text'])
test_df = pd.read_csv(test_file_path, sep='\t', names=['Classification', 'Text'])

In [86]:
## Mapping the data

x_train = train_df.drop('Classification', axis=1)
y_train = train_df.drop('Text', axis=1).astype('category')

x_test = test_df.drop('Classification', axis=1)
y_test = test_df.drop('Text', axis=1).astype('category')

In [87]:
### encoding the spam and ham into [0,1]

y_train =  y_train['Classification'].cat.codes
y_test = y_test['Classification'].cat.codes

In [88]:
## iterate over column (text) and encode raw text
VOCAB_SIZE = 1000
for i in x_train:
  encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
      max_tokens=VOCAB_SIZE)
  encoder.adapt(np.array(x_train[i]))

In [89]:
for j in x_test:
  encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
      max_tokens=VOCAB_SIZE)
  encoder.adapt(np.array(x_test[j]))

In [90]:
vocab = np.array(encoder.get_vocabulary())

In [91]:
### encode the text into indexes (y_train)
for i in x_train:
  data_train = x_train[i]
  encoded_x_train = encoder(data_train)[:3].numpy()


In [92]:
### encode the text into indexes (y_train)
for i in x_test:
  data_test = x_test[i]
  encoded_x_test = encoder(data_test)[:3].numpy()

In [93]:
for n in range(3):
  print("Original: ", data_train[n])
  print("Round-trip: ", " ".join(vocab[encoded_x_train[n]]))
  print()

Original:  ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.
Round-trip:  [UNK] [UNK] [UNK] a bad [UNK] about u [UNK] i dont like u right now i didnt know anything about [UNK] night but i guess im up for it                                                                                                                                              

Original:  you can never do nothing
Round-trip:  you can never do nothing                                                                                                                                                                      

Original:  now u sound like manky scouse boy steve,like! i is travelling on da bus home.wot has u inmind 4 recreation dis eve?
Round-trip:  now u [UNK] like [UNK] [UNK] boy [UNK] i is [UNK] on da bus [UNK] has u [UNK] 4 [UNK] dis [UNK]                                                                             

In [94]:
#### creat the model

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [95]:
# predict on a sample text without padding.
sample_text = ("how are you doing today?")
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[0.00554779]


In [96]:
# predict on a sample text with padding
padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

[0.00554779]


In [97]:
###compiling the model

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [98]:
model.fit(x=x_train,y=y_train,epochs=10,validation_data=(x_test, y_test), verbose=1)

Epoch 1/10
131/131 [==============================] - 11s 38ms/step - loss: 0.6010 - accuracy: 0.8660 - val_loss: 0.4835 - val_accuracy: 0.8657
Epoch 2/10
131/131 [==============================] - 3s 20ms/step - loss: 0.3830 - accuracy: 0.8660 - val_loss: 0.2460 - val_accuracy: 0.8657
Epoch 3/10
131/131 [==============================] - 3s 20ms/step - loss: 0.1677 - accuracy: 0.9129 - val_loss: 0.1149 - val_accuracy: 0.9670
Epoch 4/10
131/131 [==============================] - 3s 20ms/step - loss: 0.0971 - accuracy: 0.9718 - val_loss: 0.0840 - val_accuracy: 0.9741
Epoch 5/10
131/131 [==============================] - 3s 20ms/step - loss: 0.0700 - accuracy: 0.9777 - val_loss: 0.0675 - val_accuracy: 0.9763
Epoch 6/10
131/131 [==============================] - 3s 20ms/step - loss: 0.0559 - accuracy: 0.9825 - val_loss: 0.0583 - val_accuracy: 0.9799
Epoch 7/10
131/131 [==============================] - 3s 20ms/step - loss: 0.0461 - accuracy: 0.9866 - val_loss: 0.0531 - val_accuracy: 0.984

In [99]:
pred_text = "how are you doing today?"
predictions = model.predict(np.array([sample_text]))
print("prediction was",predictions[0][0])

prediction was -5.6187515


In [114]:
def predict_message(pred_text):
  result = model.predict([pred_text])
  if result >= 0.0:
    prediction  = [result[0][0],'spam']
  else:
    prediction  = [result[0][0],'ham']

  return (prediction)

pred_text = "how are you doing today"

prediction = predict_message(pred_text)
print(prediction)

[-5.6187515, 'ham']


In [117]:
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
